In [25]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 73 not upgraded.


In [26]:
import cv2
import numpy as np
import cv2
import os
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
map_data = {}

def add_map(map_id, value_id, color, lower_bound, upper_bound):
    map_data[map_id][value_id] = {
        "color": color,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound
    }

In [28]:
image_files = [f for f in os.listdir(path="/kaggle/input/dv-pe/accuracy_images") if f.endswith(('.png'))]

map_data_path = "/kaggle/working/map_data.txt"

def num_there(s):
    return any(i.isdigit() for i in s)

In [29]:
def mask(image):
    x, y, w, h = 188, 17, 283, 32

    x = x - (w//2)
    y = y - (h//2)

    # Create a mask
    mask = np.ones_like(image) * 255  # White mask

    # Draw a black rectangle on the mask to cover the region outside the bounding box
    cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 0, 0), thickness=cv2.FILLED)

    # Bitwise AND operation to apply the mask
    image = cv2.bitwise_and(image, mask)

    # Display the original image and the masked image using matplotlib
    plt.figure(figsize=(10, 5))

    # plt.subplot(1, 2, 1)
    # plt.imshow(cv2.cvtColor(images, cv2.COLOR_BGR2RGB))
    # plt.title("Original Image")
    # plt.axis("off")

#     plt.subplot(1, 2, 2)
#     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#     plt.title("Masked Image")
#     plt.axis("off")

#     plt.show()
    
    return image

In [ ]:
for image in image_files:
    image_dir = "/kaggle/input/dv-pe/accuracy_images"
    image_path = os.path.join(image_dir, image)
    # Argument Parser
    args = {"image": image_path, "pre_processor": "thresh"}  # Replace with your image path

    # print(map_name)
    # Reading the image with text
    images = cv2.imread(args["image"])
    image_rgb = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)

    # Converting to grayscale image
    gray = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)

    # performing preprocessing techniques of threshing and/or blurring.
    if args["pre_processor"] == "thresh":
        gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    if args["pre_processor"] == "blur":
        gray = cv2.medianBlur(gray, 3)

    # adding image to memory
    # filename = "{}.png".format(os.getpid())
    # cv2.imwrite(filename, gray)
    # text = pytesseract.image_to_string(Image.open(filename))
    # print(text)

    images = mask(images)
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Masked Image")
    plt.axis("off")

    plt.show()

    filename = "{}.png".format(os.getpid())
    # print(filename)
    cv2.imwrite(filename, gray)

    data = pytesseract.image_to_data(Image.open(filename), output_type=pytesseract.Output.DICT)

    # Looping over the detected text and the bounding boxes
    for i in range(len(data["text"])):
        value_id = 0
        if data["text"][i].strip() != "":
            x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
            # cv2.rectangle(images, (x, y), (x + w, y + h), (0, 255, 0), 2)

            text = data["text"][i]
            # print(f"Text: {text}, Bounding Box: {x}, {y}, {w}, {h}")

            if num_there(text)==1:
                value_id += 1
                # print(type(text))
                # print(f"Text: {text}, Bounding Box: {x}, {y}, {w}, {h}")
                # cv2.rectangle(images, (x, y), (x + w, y + h), (0, 255, 0), 2)

                x1 = x
                x2 = x
                y1 = y + h // 2
                y2 = y + h // 2 - 1

                while all(images[y1, x1]==images[y2, x2]):
                    x2-=1
                x2-=10

                color = images[y2,x2]
                # print(color)
                # cv2.rectangle(images, (x2, y2), (x2+5, y2+5), (0, 255, 0), 2)

                values = text.split("-")
                # print(values)

                if len(values)==1:
                    lower_bound = values[0]
                    upper_bound = values[0]
                else:
                    lower_bound = values[0]
                    upper_bound = values[1]

                blue, green, red = color
                print(f"Blue: {blue}, Green: {green}, Red: {red}")

                # add_map(map_id, value_id, (blue, green, red), lower_bound, upper_bound)
                # print(map_data)

                map_name = "map_"+image[4:-4]
                if map_name in map_data:
                    map_data[map_name][value_id] = {
                        "color": (blue, green, red),
                        "lower_bound": lower_bound,
                        "upper_bound": upper_bound,
                    }
                else:
                    map_data[map_name] = {
                        value_id: {
                            "color": (blue, green, red),
                            "lower_bound": lower_bound,
                            "upper_bound": upper_bound,
                        }
                    }

                with open(map_data_path, 'a') as f2:
                    f2.write(map_name+":")
                    f2.write(str(value_id)+":(")
                    f2.write(str(blue)+","+str(green)+","+str(red)+"),")
                    f2.write(str(lower_bound)+",")
                    f2.write(str(upper_bound))
                    f2.write("\n---------------------\n")
#                 print(map_data)

                # color = images[y,x-25] # have to edit this to get the exact colour.
                # print(f'Color at ({x}, {y}): {color}')

os.remove(filename)
# plt.imshow(cv2.cvtColor(images, cv2.COLOR_BGR2RGB))